# Semeval 2017 Task 4: Sentiment Analysis in Twitter
## Subtask C: Topic-Based Message Polarity Classification
### Given a message, classify sentiment conveyed in the tweet towards the topic on a five-point scale (-2,-1,0,1,2).

https://competitions.codalab.org/competitions/15937

In [1]:
import os
import numpy as np
import pandas as pd

from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.utils.np_utils import to_categorical

import preprocessor as p
p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.EMOJI,p.OPT.SMILEY,p.OPT.NUMBER,p.OPT.HASHTAG)

#pd.set_option('display.max_columns', 100)
#pd.set_option('display.max_rows', 100)

Using Theano backend.


## Загрузка и первичный анализ данных

* Выгрузим тренировочные и тестовые данные и посмотрим на них.

In [2]:
with open("train.txt") as f:
    train_data = f.readlines()
with open("test.txt") as f:
    test_data = f.readlines()

In [3]:
train_data = pd.DataFrame([i.split('\t') for i in train_data])
test_data = pd.DataFrame([i.split('\t') for i in test_data])

train_data.columns = ['id','topic','polarity','text']
test_data.columns = ['id','topic','polarity','text']

train_data = train_data[['id','topic','text','polarity']]
test_data = test_data[['id','topic','text','polarity']]

In [4]:
train_data.head()

,id,topic,text,polarity
0,628949369883000832,@microsoft,dear @Microsoft the newOoffice for Mac is grea...,-1
1,628976607420645377,@microsoft,@Microsoft how about you make a system that do...,-2
2,629023169169518592,@microsoft,I may be ignorant on this issue but... should ...,-1
3,629179223232479232,@microsoft,"Thanks to @microsoft, I just may be switching ...",-1
4,629186282179153920,@microsoft,If I make a game as a #windows10 Universal App...,0


In [5]:
test_data.head()

,id,topic,text,polarity
0,681563394940473347,amy schumer,@MargaretsBelly Amy Schumer is the stereotypic...,-1
1,675847244747177984,amy schumer,@dani_pitter I mean I get the hype around JLaw...,-1
2,672827854279843840,amy schumer,Amy Schumer at the #GQmenoftheyear2015 party i...,-1
3,671502639671042048,amy schumer,"""Amy Schumer may have brought us Trainwreck, b...",-1
4,677359143108214784,amy schumer,I just think that sports are stupid &amp;anyon...,-1


- Как видно, данные содержат `id, topic, text и polarity`.   
- Для классификации нам не нужны поля `id` - так как твит уже выгружен, и `topic` в этом подходе использоваться не будет.  
- Кроме того, для удобства дальнейшей категоризации значений целевой переменной сделаем категории от 0 до 4 вместо -2 до 2

In [6]:
train_data = train_data.drop(['id','topic'], axis = 1)
test_data = test_data.drop(['id','topic'], axis = 1)

train_data['polarity'] = [int(x) + 2 for x in train_data['polarity']]
test_data['polarity'] = [int(x) + 2 for x in test_data['polarity']]

In [7]:
train_data.head()

,text,polarity
0,dear @Microsoft the newOoffice for Mac is grea...,1
1,@Microsoft how about you make a system that do...,0
2,I may be ignorant on this issue but... should ...,1
3,"Thanks to @microsoft, I just may be switching ...",1
4,If I make a game as a #windows10 Universal App...,2


In [8]:
test_data.head()

,text,polarity
0,@MargaretsBelly Amy Schumer is the stereotypic...,1
1,@dani_pitter I mean I get the hype around JLaw...,1
2,Amy Schumer at the #GQmenoftheyear2015 party i...,1
3,"""Amy Schumer may have brought us Trainwreck, b...",1
4,I just think that sports are stupid &amp;anyon...,1


- Теперь, когда данные приведены к единому виду, необходимо выполнить предобработку текста: привести текст к нижнему регистру, очистить его от всего лишнего - спец символов, чисел, ссылок и зарезервированных слов.

## Предобработка текста

* Возьмем твит из тренировочной выборки и посмотрим на него.

In [9]:
train_data['text'][7]

'@Microsoft 2nd computer with same error!!! #Windows10fail Guess we will shelve this until SP1! http://t.co/QCcHlKuy8Q\n'

* Текст необходимо очистить. Для этого воспользуемся библиотекой для предобработки твитов - https://github.com/s/preprocessor  
Ее особенность в том, что она позволяет удалять ссылки, хэштеги, упоминания и т.д. 
Код применения ниже:

In [10]:
p.clean(train_data['text'][7].lower())

'nd computer with same error!!! guess we will shelve this until sp1!'

* Теперь воспользуемся токенайзером, который очистит текст от оставшихся ненужных символов и соединим получившиеся токены в предложения

In [11]:
' '.join(CountVectorizer().build_tokenizer()(p.clean(train_data['text'][7].lower())))

'nd computer with same error guess we will shelve this until sp1'

* Теперь сделаем то же самое со всеми данными

In [12]:
train_data['text'] = train_data['text'].apply(lambda t: ' '.join(CountVectorizer().build_tokenizer()(p.clean(t.lower()))))
test_data['text'] = test_data['text'].apply(lambda t: ' '.join(CountVectorizer().build_tokenizer()(p.clean(t.lower()))))

In [13]:
train_data.head()

,text,polarity
0,dear the newooffice for mac is great and all b...,1
1,how about you make system that doesn eat my fr...,0
2,may be ignorant on this issue but should we ce...,1
3,thanks to just may be switching over to,1
4,if make game as universal app will owners be a...,2


In [14]:
test_data.head()

,text,polarity
0,amy schumer is the stereotypical st world laci...,1
1,mean get the hype around jlaw may not like her...,1
2,amy schumer at the party in dress we pretty mu...,1
3,amy schumer may have brought us trainwreck but...,1
4,just think that sports are stupid amp anyone w...,1


## Частотное представление твита

* Далее, представим твиты в виде последовательности чисел. Для этого, воспользуемся Tokenizer'ом
* Установим максимальное количество слов (по частоте использования) равным 25000 и заполним токенайзер твитами.

In [15]:
max_features = 25000

tokenizer = Tokenizer(max_features)
tokenizer.fit_on_texts(list(train_data['text']) + list(test_data['text']))
print('Уникальных слов - ', len(tokenizer.word_index))

Уникальных слов -  27240


* Приведем твиты в числовые последовательности

In [16]:
train_data['text'] = tokenizer.texts_to_sequences(train_data['text'])
test_data['text'] = tokenizer.texts_to_sequences(test_data['text'])

* Установим максимальную длину последовательности
* Обрежем длинные и дополним короткие

In [17]:
maxlen = 50

X_train = pad_sequences(list(train_data['text']), maxlen)
X_test = pad_sequences(list(test_data['text']), maxlen)

* Проведем категоризацио значений целевой переменной

In [18]:
y_train = to_categorical(train_data['polarity'])
y_test = to_categorical(test_data['polarity'])

## Создание модели рекуррентной нейронной сети с LSTM

Так как тренировочная выборка - всего 10000 твитов, в качестве входного слоя возьмем уже предобученный на **твитах** слой векторного представления слов. Взять его можно отсюда - https://nlp.stanford.edu/projects/glove/

* *max_features* - количество слов
* *max_len* - длина последовательности твита
* *embedding_dim* - размерность векторного представления слова

In [19]:
max_features = max_features
max_len = maxlen
embedding_dim = 200

* Подготовка слоя векторного представления слов

In [20]:
BASE_DIR = '/Users/dev/Desktop/SemEval-2017/'
GLOVE_DIR = BASE_DIR + 'glove.twitter.27B'
print('Indexing word vectors.')

embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.twitter.27B.200d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

# prepare embedding matrix
embedding_matrix = np.zeros((max_features, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed

Indexing word vectors.
Found 1193514 word vectors.


#### Модель
* model = Sequential() - чтобы принимала на вход последовательности слов  

1) Слой Embedding, содержит веса векторного представления слов, настройки говорят о том, что в словаре 25000 разных фичей, а сетке ждать последовательности из не более, чем 50 слов  
2) Далее два слоя LSTM, каждый из которых отдаёт на выход тензор размерность batch_size / length of a sequence / units in LSTM, а второй отдаёт матрицу batch_size / units in LSTM. Чтобы второй понимал первого, выставлен флаг return_sequences=True  
3) Слой Dropout отвечает за переобучение. Он обнуляет случайную половину фичей и мешает коадаптации весов в слоях  
4) Dense-слой взвешенно суммирует компоненты входного вектора  
5) Последний слой выдает значение от 0 до 4 и используется функция активации softmax 

https://habrahabr.ru/company/dca/blog/274027/

In [28]:
model = Sequential()
#model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(Embedding(max_features,
                    embedding_dim,
                    weights=[embedding_matrix],
                    input_length=maxlen,
                    trainable=False))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics = ['accuracy'])

* Обучаем модель

In [31]:
model.fit(
    X_train, y_train, 
    batch_size = 32, 
    epochs = 1,
    validation_data = (X_test, y_test)
)

Train on 10000 samples, validate on 20632 samples
Epoch 1/1
10000/10000 [==============================] - 49s - loss: 0.9227 - acc: 0.6049 - val_loss: 0.9425 - val_acc: 0.5579

* Выведем предсказанные метки и посмотрим на показатели точности, полноты и f1-score. 

In [33]:
predictions = model.predict_classes(X_test)

20576/20632 [============================>.] - ETA: 0s 

In [34]:
print(metrics.classification_report(predictions, test_data['polarity']))

             precision    recall  f1-score   support

          0       0.04      0.36      0.07        14
          1       0.33      0.41      0.37      1794
          2       0.47      0.64      0.54      7444
          3       0.77      0.53      0.63     11362
          4       0.01      0.28      0.03        18

avg / total       0.62      0.56      0.57     20632



*Код для сохранения решения в необходимом формате*

In [35]:
predict = [str(x-2) for x in predictions]

with open("test.txt") as f:
    test = f.readlines()

for i in range (0,len(test)):
    test[i] = test[i].split('\t')
    test[i].pop(-1)
    test[i][-1] = predict[i]
    test[i] = '\t'.join(test[i])
    
file = open('newfile.txt', 'w')
for i in test:
    file.write(i + '\n')
file.close()